# Pyg4omety
* **Stewart Boogert** (my first ever notebook talk, be kind!!) 
* Laurie Nevay
* William Shields
* Andrey Abramov
* Luigi Pertoldi
* Stuart Walker

## Introduction

Usage notes 
  * This notebook requires that the requirements for VTK openGL rendering are installed. This is possible in docker and most modern linuxes. For those users where there installation is not working correctly. Screenshots of the output are included in the notebook. VTK embedded in jupyter is actively being developed, but the authors did not have sufficient time for testing
  * Each section does not depend significantly on the previous section.
  * Some of the output files generated in one section are used in subsequent sections

## Creating a simple GDML/Geant4 geometry

### Create a registry 

GDML really is a gloabl namespace for names of geometry objects. This global namescape needs to managed to avoid name clashes etc. This is very important when looking at geometry compositing

In [18]:
from pyg4ometry import geant4 as _g4
from pyg4ometry import visualisation as _vi
from pyg4ometry import gdml as _gdml

In [19]:
reg = _g4.Registry()
reg.clear()

### Create a few NIST materials

In [20]:
worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
boxMaterial   = _g4.nist_material_2geant4Material("G4_Au", reg)

### Create some solids

In [21]:
worldSolid = _g4.solid.Box("ws", 100, 100, 100, reg, "mm")
boxSolid   = _g4.solid.Box("bs", 10, 10, 10, reg, "mm")

### Create logical and physical volumes

In [22]:
worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
boxLogical   = _g4.LogicalVolume(boxSolid, boxMaterial, "boxlogical", reg)
boxPhysical  = _g4.PhysicalVolume([0, 0, 0], [0, 0, 0], boxLogical, "boxPhysical1", worldLogical, reg)
reg.setWorld(worldLogical.name)

### Visualise

In [23]:
v = _vi.VtkViewerNew()
v.addLogicalVolume(reg.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

![Basic example screen shot](./screenshots/basicScreenshot.png)

### Write to GDML file

In [8]:
w = _gdml.Writer()
w.addDetector(reg)
w.write("box.gdml")

## Parameterised geometry

GDML supports variables which are evaluated when the geometry is used. 

In [24]:
from pyg4ometry import geant4 as _g4
from pyg4ometry import visualisation as _vi
from pyg4ometry import gdml as _gdml
reg = _g4.Registry()
reg.clear()

In [25]:
worldSize = _gdml.Constant("boxSize", 100, reg)
boxSize   = _gdml.Constant("worldSize", 10, reg)

In [26]:
# materials 
worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
boxMaterial   = _g4.nist_material_2geant4Material("G4_Au", reg)

# solids 
worldSolid = _g4.solid.Box("ws", worldSize, worldSize, worldSize, reg, "mm")
boxSolid = _g4.solid.Box("bs", boxSize, boxSize, boxSize, reg, "mm")

# structure
worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
boxLogical   = _g4.LogicalVolume(boxSolid, boxMaterial, "boxlogical", reg)
boxPhysical  = _g4.PhysicalVolume([0, 0, 0], [0, 0, 0], boxLogical, "boxPhysical1", worldLogical, reg)

# visualise
reg.setWorld(worldLogical.name)
v = _vi.VtkViewerNew()
v.addLogicalVolume(reg.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

### Calculations using parameters

In [27]:
boxSize = worldSize/20

In [28]:
boxSize

Constant : var_boxSize_div_20.000000000000000 = expr_var_boxSize_div_20.000000000000000 : (boxSize) / (20.000000000000000)

In [30]:
import numpy as _np
reg = _g4.Registry()
reg.clear()

worldSize = _gdml.Constant("boxSize", 200, reg)
boxSize = worldSize/20

# materials 
worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
boxMaterial   = _g4.nist_material_2geant4Material("G4_Au", reg)

# solids 
worldSolid = _g4.solid.Box("ws", worldSize, worldSize, worldSize, reg, "mm")
boxSolid = _g4.solid.Box("bs", boxSize, boxSize, boxSize, reg, "mm")

ringRadius = _gdml.Constant("ringSize", 50, reg)

# structure
worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
boxLogical   = _g4.LogicalVolume(boxSolid, boxMaterial, "boxlogical", reg)

for i in range(0,20,1) :
    ic = _gdml.Constant("i_"+str(i),i/20,reg)
    phi = 2*_np.pi*ic
    x = ringRadius*_gdml.cos(phi)
    y = ringRadius*_gdml.sin(phi)
    
    boxPhysical  = _g4.PhysicalVolume([0, 0, -phi], [x, y, 0], boxLogical, "boxPhysical_"+str(i), worldLogical, reg)

# visualise
reg.setWorld(worldLogical.name)
v = _vi.VtkViewerNew()
v.addLogicalVolume(reg.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

Important to note that Constants are written to the GDML file

In [117]:
w = _gdml.Writer()
w.addDetector(reg)
w.write("boxRing.gdml")

In [32]:
!cat boxRing.gdml

<?xml version="1.0" ?>
<gdml xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://service-spi.web.cern.ch/service-spi/app/releases/GDML/schema/gdml.xsd">
	<define>
		<constant name="boxSize" value="200.000000000000000"/>
		<constant name="ringSize" value="50.000000000000000"/>
		<constant name="i_0" value="0.000000000000000"/>
		<constant name="i_1" value="0.050000000000000"/>
		<constant name="i_2" value="0.100000000000000"/>
		<constant name="i_3" value="0.150000000000000"/>
		<constant name="i_4" value="0.200000000000000"/>
		<constant name="i_5" value="0.250000000000000"/>
		<constant name="i_6" value="0.300000000000000"/>
		<constant name="i_7" value="0.350000000000000"/>
		<constant name="i_8" value="0.400000000000000"/>
		<constant name="i_9" value="0.450000000000000"/>
		<constant name="i_10" value="0.500000000000000"/>
		<constant name="i_11" value="0.550000000000000"/>
		<constant name="i_12" value="0.600000000000000"/>
		<constant name=

&#x26a0;&#xfe0f; Care needs to be taken in not using normal number types with modifying Constants, Defines and Quantities and only the appropriate class &#x26a0;&#xfe0f;

![Example parametrised construction](./screenshots/paramScreenShot.png)

## Loading, understanding and  editing geometry

Objects
 * solids, 
 * materials 
 * optical surfaces
 * logical volumes etc) 
 
can be stored in flat dictionaries as key - value pairs. The structure (logical volume - physical volume hierachy in stored as a multiply linked list within the logical and physics volume objects

In [50]:
from pyg4ometry import gdml as _gdml

In [51]:
r = _gdml.Reader("box.gdml")
reg = r.getRegistry()

In [52]:
reg.solidDict

{'ws': Box : ws Expression : ws_pX = expr_ws_pX : 100 Expression : ws_pY = expr_ws_pY : 100 Expression : ws_pZ = expr_ws_pZ : 100,
 'bs': Box : bs Expression : bs_pX = expr_bs_pX : 10 Expression : bs_pY = expr_bs_pY : 10 Expression : bs_pZ = expr_bs_pZ : 10}

In [53]:
reg.materialDict

{'G4_H_1': <Isotope: G4_H_1>,
 'G4_H_2': <Isotope: G4_H_2>,
 'G4_H': <Element: G4_H>,
 'G4_Au': <Element: G4_Au>,
 'G4_Galactic': <Material: G4_Galactic>,
 'Material_G4_Au': <Material: Material_G4_Au>}

In [56]:
print("Logical volumes")    
for lvKey in reg.logicalVolumeDict :
    lv = reg.logicalVolumeDict[lvKey]
    print(lvKey, lv.type, lv.solid)
    
print("Physical volumes")    
for pvKey in reg.physicalVolumeDict :
    pv = reg.physicalVolumeDict[pvKey]
    print(pvKey, pv.type, pv.position.eval(), pv.rotation.eval())

Logical volumes
boxlogical logical Box : name=bs x=10.0 y=10.0 z=10.0
worldLogical logical Box : name=ws x=100.0 y=100.0 z=100.0
Physical volumes
boxPhysical1 placement [0.0, 0.0, 0.0] [0.0, 0.0, 0.0]


In [57]:
worldLogical = reg.getWorldVolume()

So the flattened storage is not appropriate for the geometry hierachy. The entire geometrical structure of a "detector" can be accessed via the **world logical volume** ooping over the daughter volumes of the world

In [61]:
for dv in worldLogical.daughterVolumes: 
    print(dv.name)

boxPhysical1


Of course a physical volume has a logical volume. A logical volume has a list of daughter physical volumes. The hierachy looks some thing like this

* logicalVolume (world) 
  * physicalVolume (daughter1) 
    * logicalVolume 
        * physicalVolume (daughter1-2)
            * logicalVolume
        * physicalVolume (daughter1-2)
            * logicalVolume

In [62]:
worldLogical.daughterVolumes[0].logicalVolume

Logical volume : boxlogical Box : name=bs x=10.0 y=10.0 z=10.0 Material_G4_Au

The LV-PV tree (also called a scene tree in computer graphics) terminates on a logical volume. So for viewing and geometric algorithms the logical volume contains two represetnations of the solid, parametrised and tesselated mesh

In [37]:
worldLogical.solid

Box : ws Expression : ws_pX = expr_ws_pX : 100 Expression : ws_pY = expr_ws_pY : 100 Expression : ws_pZ = expr_ws_pZ : 100

In [40]:
worldLogical.mesh
worldLogical.mesh.getBoundingBox()

&#x26a0;&#xfe0f; **When editing the in memory geometry structure (changing solids or placements etc). 
The geometry needs to be explicitly remeshed to keep the geometry consistent.** &#x26a0;&#xfe0f;

## Importing other formats

Simple function to display a solid (nothing new here)

In [8]:
def displaySolid(solid) :
    reg = _g4.Registry()
    
    # materials 
    worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
    material   = _g4.nist_material_2geant4Material("G4_Au", reg)

    # solids 
    worldSolid = _g4.solid.Box("worldSolid", 25, 25, 25, reg, "mm")

    # structure
    worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
    logical   = _g4.LogicalVolume(solid, material, "boxlogical", reg)
    physical  = _g4.PhysicalVolume([0, 0, 0], [0, 0, 0], logical, "boxPhysical1", worldLogical, reg)

    # visualise
    reg.setWorld(worldLogical.name)
    v = _vi.VtkViewerNew()
    v.addLogicalVolume(reg.getWorldVolume())
    v.buildPipelinesAppend()
    v.view(interactive=True)

In [9]:
from pyg4ometry import stl as _stl

In [10]:
stl = _stl.Reader("./utah_teapot.stl")
stlSolid = stl.getSolid()
displaySolid(stlSolid)

![Example STL file load](./screenshots/stlScreenshot.png)

In [11]:
from pyg4ometry import pyoce as _pyoce
from pyg4ometry import convert as _con

cad        = _pyoce.Reader("./bodies.step")
freeShapes = cad.freeShapes()
label = freeShapes.Value(1)
shape = cad.shapeTool.GetShape(label)

reg = _g4.Registry()
cadSolid = _con.oceShape_Geant4_Tessellated("cadSolid",shape,reg)
displaySolid(cadSolid)

![Example a CAD (STEP) file load](./cadScreenShot.png)

## Compositing geometry

In general if you have loaders from different formats, it is relatively simple of composite 
geometry together in a single in-memory representation. The problem comes with potential **name** clashes. To illustrate this we'll load our simple box example twice 

In [16]:
reader1 = _gdml.Reader("box.gdml")
reader2 = _gdml.Reader("box.gdml")
registry1 = reader1.getRegistry()
registry2 = reader2.getRegistry()

In [17]:
registry3 = _g4.Registry()
registry3.clear()

logicalVolume1 = registry1.getWorldVolume()
logicalVolume2 = registry2.getWorldVolume()

worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", registry3)
worldSolid    = _g4.solid.Box("worldSolid", 500, 500, 500, registry3, "mm")
worldLogical  = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", registry3)

physicalVolume1 = _g4.PhysicalVolume([0,0, 0.5],[-150, 0, 0], logicalVolume1, "file1Physical", worldLogical, registry3) 
boxPhysical     = _g4.PhysicalVolume([0, 0, -0.5], [ 150, 0, 0], logicalVolume2, "file2Physical", worldLogical, registry3)

registry3.addVolumeRecursive(logicalVolume1)
registry3.addVolumeRecursive(logicalVolume2);
registry3.setWorld(worldLogical.name)

v = _vi.VtkViewerNew()
v.addLogicalVolume(registry3.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

Ok lets take a look at the GDML output for the merged geometry

In [90]:
w = _gdml.Writer()
w.addDetector(registry3)
w.write("merge.gdml")
! cat merge.gdml

![Example compositing two geometries](./screenshots/compositeScreenShot.png)

## Overlap checking

So given a set of surface meshes it is possible to find the intersections between these meshes. This is exceptionally useful for debugging general problems with the simulation geomtery

In [ ]:
from pyg4ometry import geant4 as _g4
from pyg4ometry import visualisation as _vi
from pyg4ometry import gdml as _gdml
reg = _g4.Registry()
reg.clear()

worldSize = _gdml.Constant("boxSize", 100, reg)
boxSize   = _gdml.Constant("worldSize", 50, reg)

# materials 
worldMaterial = _g4.nist_material_2geant4Material("G4_Galactic", reg)
boxMaterial   = _g4.nist_material_2geant4Material("G4_Au", reg)

# solids 
worldSolid = _g4.solid.Box("ws", worldSize, worldSize, worldSize, reg, "mm")
boxSolid = _g4.solid.Box("bs", boxSize, boxSize, boxSize, reg, "mm")

# structure
worldLogical = _g4.LogicalVolume(worldSolid, worldMaterial, "worldLogical", reg)
boxLogical   = _g4.LogicalVolume(boxSolid, boxMaterial, "boxlogical", reg)
boxPhysical1  = _g4.PhysicalVolume([0, 0, 0], [-15+25, 0, 0], boxLogical, "boxPhysical1", worldLogical, reg)
boxPhysical2  = _g4.PhysicalVolume([0, 0, 0], [15+25, 0, 0], boxLogical, "boxPhysical2", worldLogical, reg)

worldLogical.checkOverlaps(True)

# visualise
reg.setWorld(worldLogical.name)
v = _vi.VtkViewerNew()
v.addLogicalVolume(reg.getWorldVolume())
v.buildPipelinesAppend()
v.view(interactive=True)

![VTK visualisation of overlaps](./screenshots/overlapScreenShot.png)

&#x26a0;&#xfe0f; **Overlap detection is an approximation as the mesh representing the solid is an approximation. Clearly a better approximation can be achieved if a denser mesh is used, but at an increased computational cost. Not using techniques based on apporximate meshes because potentiall accuracy issues then prevents rapid approximate methods, but user mush be awear of this** &#x26a0;&#xfe0f;

## Converting to other codes (FLUKA)

Having an API for geometry allows for some impressive applications. Interoperation between Geant4 and infinite half space representations of geometry (H-Rep) has been a challenge. With a few algorithms from CGAL (for example 2D decomposition to union of convex hulls) can write simple converter.

Here have a similar API to FLUKA geometry and FLUKA registry and develop algorithm to convert between representations.

&#x26a0;&#xfe0f; **Translating between GDML and FLUKA is relatively simple, apart from twisted solids. Going in the other direction (FLUKA to GDML) is rather more challenging and although supported by pyg4ometry is not elegant or robust.** &#x26a0;&#xfe0f;

In [110]:
from pyg4ometry import gdml as _gdml
from pyg4ometry import convert as _convert
from pyg4ometry import fluka as _fluka

In [118]:
r = _gdml.Reader("boxRing.gdml")

In [119]:
freg = _convert.geant4Reg2FlukaReg(r.getRegistry())

In [120]:
w = _fluka.Writer()
w.addDetector(freg)
w.write("boxRing.inp")

In [121]:
!cat boxRing.inp

FREE
GEOBEGIN , , , , , , , COMBNAME
    0    0
RPP BLKBODY -20.0 20.0 -20.0 20.0 -20.0 20.0
* worldLogical ws
RPP B000001 -10.0 10.0 -10.0 10.0 -10.0 10.0
$start_transform T0002
* boxPhysical_0 bs
RPP B000201 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0003
* boxPhysical_1 bs
RPP B000301 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0004
* boxPhysical_2 bs
RPP B000401 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0005
* boxPhysical_3 bs
RPP B000501 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0006
* boxPhysical_4 bs
RPP B000601 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0007
* boxPhysical_5 bs
RPP B000701 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0008
* boxPhysical_6 bs
RPP B000801 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0009
* boxPhysical_7 bs
RPP B000901 -0.5 0.5 -0.5 0.5 -0.5 0.5
$end_transform
$start_transform T0010
* boxPhysical_8 bs
RPP B001001 -0.5 0.5 -0.5 0.5 -0.5

![Flair FLUKA GUI interface](./screenshots/flairScreenShot.png)

This is Flair a GUI interface to FLUKA input files. 